## Tiltmeter Overview Plot

### Import Libraries

In [1]:
import os
import obspy
import matplotlib.pyplot as plt
import pandas as pd

from numpy import arange, linspace, sqrt, diff, nan, gradient, nanmax
from pandas import read_csv, DataFrame
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from numpy.ma import filled, isMaskedArray, masked
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from obspy import UTCDateTime
from pathlib import Path

from andbro__querrySeismoData import __querrySeismoData
from andbro__get_seconds import __get_seconds
from andbro__readYaml import __readYaml
from andbro__get_timeaxis import __get_timeaxis

### Define Functions

In [2]:
def __makeplot(pt, bt, config):
    
    fig, axes = plt.subplots(3,2, figsize=(15,5), sharex=True)
    
    for i in range(3):
        
        axes[i,0].plot(pt[i].times()/60, pt[i].data)
        axes[i,1].plot(bt[i].times()/60, bt[i].data)
        
        axes[i,0].grid()
        axes[i,1].grid()
        
    axes[2,0].set_xlabel("Time (min)")
    axes[2,1].set_xlabel("Time (min)")
    
    axes[0,0].set_ylabel("$\Omega$ (rad)")
    axes[1,0].set_ylabel("$\Omega$ (rad)")
    axes[2,0].set_ylabel("Temp (°C)")
    
    axes[0,1].set_ylabel("$\Omega$ (rad)")
    axes[1,1].set_ylabel("$\Omega$ (rad)")
    axes[2,1].set_ylabel("Temp (°C)")
    
    return fig 

### Configurations

In [3]:
## configurations
config={}

path = "/home/andbro/Desktop/tiltmeter/"
# path2 = '/home/andbro/Desktop/tiltmeter/WETTER/*ex'

config['save'] = False

config['seed_id'] = ["BW.TROMY..MA*", "BW.DROMY..LA*"]
config['repository'] = 'george'
config['datapath'] =  None
config['name'] = 'tiltmeter'
config['set_filter'] = 'n'


## tiltmeter configurations
try:
    confTilt = __readYaml('/home/brotzer/Documents/ROMY/','tiltmeter.conf')
except:
    confTilt = __readYaml('./','tiltmeter.conf')

confPT = confTilt['PT']
confBT = confTilt['BT']



In [4]:
# config['tend'] = UTCDateTime.now() - 10*60
# config['tbeg'] = config['tend'] - 15*86400

## long period
config['tbeg'] = UTCDateTime("2021-02-15 00:00")
config['tend'] = UTCDateTime("2021-11-24 00:00")

## strong rain
# config['tbeg'] = UTCDateTime("2021-08-20 00:00")
# config['tend'] = UTCDateTime("2021-09-15 00:00")

## after reset
config['tbeg'] = UTCDateTime("2021-08-01 00:00")
config['tend'] = UTCDateTime("2021-11-20 00:00")


## gain for Temperature channel of PT was not set correctly prior to 2021-03-03
correct_settings_temp_PT = ("2021-03-05", config['tend'])

### Load Tiltmeter Data

In [5]:
pt, inv0 = __querrySeismoData(    
                            seed_id=config.get("seed_id")[0],
                            starttime=config.get("tbeg"),
                            endtime=config.get("tend"),
                            where=config.get("repository"),
                            path=config.get("datapath"),
                            restitute=False,
                            detail=None,
                            fill_value=-9999, 
                            )

pt.resample(1.0)

pt

TypeError: __querrySeismoData() got an unexpected keyword argument 'where'

In [ ]:
bt, inv0 = __querrySeismoData(    
                            seed_id=config.get("seed_id")[1],
                            starttime=config.get("tbeg"),
                            endtime=config.get("tend"),
                            where=config.get("repository"),
                            path=config.get("datapath"),
                            restitute=True,
                            detail=None,
                            fill_value=-9999, 
                            )

bt.resample(0.1)

bt

### Covert Data

In [ ]:
def __conversion(st, confBT, confPT):

    def convertTemp(trace, gain):
        Tvolt = trace.data * gain
        return 10.18 - 11.59*Tvolt + 0.3335*Tvolt**2 - 0.5316*Tvolt**3

    def convertTilt(trace, conversion, sensitivity):
        return trace.data * conversion * sensitivity
        # print( type(conversion), type(sensitivity), type(trace.data) )

    for tr in st:
        if tr.stats.channel == 'MAT':
            tr.data = convertTemp(tr, confPT['gainTemp'])
        elif tr.stats.channel == 'MAN':
            tr.data = convertTilt(tr, confPT['convPTN'], confPT['gainTilt'])
        elif tr.stats.channel == 'MAE':
            tr.data = convertTilt(tr, confPT['convPTE'], confPT['gainTilt'])

        elif tr.stats.channel == 'LAT':
            tr.data = convertTemp(tr, confBT['gainTemp'])
        elif tr.stats.channel == 'LAN':
            tr.data = convertTilt(tr, confBT['convBTN'], confBT['gainTilt'])
        elif tr.stats.channel == 'LAE':
            tr.data = convertTilt(tr, confBT['convBTE'], confBT['gainTilt'])

    print("  -> converted data")
    return st

In [ ]:
pt = __conversion(pt, confBT, confPT)

bt = __conversion(bt, confBT, confPT)

In [ ]:
fig = __makeplot(pt, bt, config);

## find corrupt periods

In [ ]:

# n1 = 125_000
# n2 = n1 + 310_000

# plt.figure()
# plt.plot(pt[0].data[n1:n2])

# pt[0].times(type="utcdatetime")[n1]
# pt[0].times(type="utcdatetime")[n2]

#### cutout maintenance work and unreliable data

In [ ]:
# ## maintenance interruption 
# maintenancePT = [
#                 ("2021-03-02 16:00", "2021-03-02 19:00"),
#                 ("2021-03-04 15:00", "2021-03-04 17:00"),
#                 ("2021-03-04 23:30", "2021-03-05 00:30"),
#                 ]

# maintenanceBT = [
#                 ("2021-03-04 13:15", "2021-03-04 16:00"),
#                 ("2021-03-09 10:45", "2021-03-09 11:30"),
#                 ]

# ## data influenced probably py dehumidifier
# corrupt_data = [
#                 ("2021-02-15 16:00", "2021-02-24 14:00" ),
#                 ("2021-03-02 14:00", "2021-03-09 17:00"),
#                ]

### Cut Corrupt Periods

In [ ]:
def __cut_corrupt_data(st, name, log, config):

    from obspy import UTCDateTime
    from numpy import nan
    
    for i in range(len(log.starttime)):
        
        val1_utc, val2_utc = UTCDateTime(log.starttime[i]), UTCDateTime(log.endtime[i])
        
        
        if val1_utc > st[0].stats.starttime and val2_utc < st[0].stats.endtime:
            print(f"cutting corrupt data: {i}...")
            
            if log.device[i] in [name, " "]:
                st0 = st.slice(UTCDateTime(config['tbeg']), val1_utc )
                st0 += st.slice(val2_utc, UTCDateTime(config['tend']))

                st0.merge(fill_value = nan)

                st = st0
        
    return st

In [ ]:
config['path_to_maintenance_log'] = "/home/brotzer/Documents/ROMY/tiltmeter/tiltmeter_maintenance.log"

tiltmeter_maintenance_log = pd.read_csv(config['path_to_maintenance_log'], header=1, names=["starttime", "endtime", "device", "remark"])


pt = __cut_corrupt_data(pt, 'pt', tiltmeter_maintenance_log, config) 
bt = __cut_corrupt_data(bt, 'bt', tiltmeter_maintenance_log, config) 

In [ ]:
tiltmeter_maintenance_log

In [ ]:
fig = __makeplot(pt, bt, config);

### Correct for LongTerm Temperature Trend

In [ ]:
correction_LT_pt = __readYaml("./", "correction_longterm_pt.yml")
correction_LT_bt = __readYaml("./", "correction_longterm_bt.yml")

## correct with regression slope and temperature
pt[0].data = pt[0].data - float(correction_LT_pt.get('slopes').get('C1')) * pt[2].data
pt[1].data = pt[1].data - float(correction_LT_pt.get('slopes').get('C2')) * pt[2].data

## correct with regression slope and temperature
bt[0].data = bt[0].data - float(correction_LT_bt.get('slopes').get('C1')) * bt[2].data
bt[1].data = bt[1].data - float(correction_LT_bt.get('slopes').get('C2')) * bt[2].data

In [ ]:
fig = __makeplot(pt, bt, config);

### Load FURT Data

In [ ]:
def __read_furt_data(config, show_raw=False):
    '''
    Load a selection of data of FURT weather station for time period
    
    
    PARAMETERS:
        - config:    configuration dictionary
        - show_raw:  bool (True/False) -> shows raw data FURT head


    RETURN:
        - dataframe
        
    '''
    
    from tqdm import tqdm
    from numpy import arange
        
        
    ## define path_to_archive -> directory with data
    if 'path_to_archive' in config.keys():
        path_to_archive = config['path_to_archive']
    else:
        path_to_archive = '/bay200/gif_online/FURT/WETTER/'
        
    ## check if path with data exists
    if not Path(path_to_archive).exists():
        print(f"  -> Path: {path_to_archive}, does not exists!")
        return    
    
    
    ## list of parameters requried in configurations
    params = ['tbeg', 'tend']
    for param in params:
        if not param in config.keys():
            print(f"ERROR: {param} not in config but required!")
            return
    
    
    ## declare empyt dataframe
    df = pd.DataFrame()
    
    ## loop over day files and load data
    for i, date in tqdm(enumerate(arange(config['tbeg'].date, (config['tend']+86400).date))):
        
        date = UTCDateTime(str(date)).date
        filename = f'FURT.WSX.D.{str(date.day).rjust(2,"0")}{str(date.month).rjust(2,"0")}{str(date.year).rjust(2,"0")[-2:]}.0000'
        
#         print(f'   reading {filename} ...')

        try:
            if show_raw:
                df0 = pd.read_csv(path_to_archive+filename)            
                print(df0.columns.tolist())
                return
            else:
                df0 = pd.read_csv(path_to_archive+filename, usecols=[0,1,10,12,13,14], names=['date', 'time', 'T', 'H', 'P','Rc'])            
            
            ## substitute strings with floats
            df0['T']  = df0['T'].str.split("=", expand=True)[1].str.split("C", expand=True)[0].astype(float)
            df0['P']  = df0['P'].str.split("=", expand=True)[1].str.split("H", expand=True)[0].astype(float)
            df0['H']  = df0['H'].str.split("=", expand=True)[1].str.split("P", expand=True)[0].astype(float)
            df0['Rc'] = df0['Rc'].str.split("=", expand=True)[1].str.split("M", expand=True)[0].astype(float)
           
            
            ## replace error indicating values (-9999, 999.9) with NaN values
            df0.replace(to_replace=-9999, value=nan, inplace=True)
            df0.replace(to_replace=999.9, value=nan, inplace=True)
            
            
            if df.empty:
                df = df0
            else: 
                df = pd.concat([df, df0])
        except:
            print(f"  -> File: {filename}, does not exists!")
   
    df.reset_index(inplace=True, drop=True)
        
    return df

In [ ]:
def __processing_FURT(data, config):
    
    filter_length = 10*config['resample']
    
    data.iloc[:,3:6] = data.iloc[:,3:6].rolling(filter_length).mean() 
    print(f"Filter: rooling mean {filter_length}!")
    
    data = data[data.index % config['resample'] == 0]
    print(f"Resampling: keep every {config['resample']}nth sample!")
    
    return data

In [ ]:
FURT = __read_furt_data(config)

In [ ]:
## set column headers
FURT = FURT[['date', 'time', 'T', 'P', 'H', 'Rc']]

## resampling 
config['resample'] = 10
FURT = __processing_FURT(FURT, config)


## check if date column consists of integers
if str(FURT['date'].dtype) == 'float64':
    FURT['date'] = FURT['date'].astype('int')

    
## add a column with day of year
FURT['doy'] = [int(UTCDateTime(f'20{str(int(dat)).rjust(6,"0")[-2:]}-{str(int(dat)).rjust(6,"0")[-4:-2]}-{str(int(dat)).rjust(6,"0")[-6:-4]}').julday) for dat in FURT.date]

In [ ]:
def __get_vector(N, E, Twin=3600):

    from numpy import median, sqrt, arctan, degrees, mod, array, append, rad2deg

    
    dt = N[0].stats.delta
    t1 = N[0].stats.starttime
    t1 = 0

    Ndata = N[0].data
    Edata = E[0].data

    vnorm, vdir, timeline = [], [], []
    while (t1 + Twin*dt) < N[0].stats.npts:


        Nmed = median(Ndata[int(t1*dt):int((t1+Twin)*dt)])
        Emed = median(Edata[int(t1*dt):int((t1+Twin)*dt)])

        vnorm.append(sqrt(Nmed**2 + Emed**2))
    #     vdir.append(mod(degrees(arctan(Nmed/Emed)), 360))
        vdir.append(rad2deg(arctan(Nmed/Emed)))

        t1 += Twin/2*dt
    
        timeline.append(t1*dt)
    
    
    time_offset = N[0].stats.starttime.hour*3600+ N[0].stats.starttime.minute*60 + N[0].stats.starttime.second
    timeline = ( array(timeline) + time_offset ) / 3600 /24 # in hours
    
    return timeline, array(vdir), array(vnorm)

bt_vtimeline, bt_vdir, bt_vnorm = __get_vector(bt.select(channel="*N"), bt.select(channel="*E"), Twin=3600)
pt_vtimeline, pt_vdir, pt_vnorm = __get_vector(pt.select(channel="*N"), pt.select(channel="*E"), Twin=3600)

### Plotting

In [ ]:
def __makeplote_overview(pt, bt, FURT):
    
    fig, ax = plt.subplots(8, 1, figsize=(15,10), sharex=True)

    font=14

    timeline_furt, ticks_furt, ticklabels_furt, text_furt = __get_timeaxis(
                                                                            dates=FURT['date'],
                                                                            times=FURT['time'],
                                                                            unit='date',
                                                                            unitmode='relative',
                                                                            dateformat='ddmmyy',
                                                                          )
    utcdatetimes = bt[0].times(type="utcdatetime")
    
    timeline, ticks, ticklabels, text = __get_timeaxis(
                                                        utcdatetime=utcdatetimes,
                                                        unit='date',
                                                        unitmode='relative',
                                                        dateformat='ddmmyy',
                                                      )    
    
    
    
    
    ax[0].plot(timeline, bt[0].data)
    
    ax[6].plot(timeline_furt, FURT['T']) 


    plt.show();
    return fig
    
# __makeplote_overview(pt, bt, FURT);

In [ ]:
def __makeplote_platform(st, FURT):
    
    N = 6
    
    fig, ax = plt.subplots(N, 1, figsize=(15,10), sharex=True)

    font=14

    timeline_furt, ticks_furt, ticklabels_furt, text_furt = __get_timeaxis(
                                                                            dates=FURT['date'],
                                                                            times=FURT['time'],
                                                                            unit='date',
                                                                            unitmode='relative',
                                                                            dateformat='ddmmyy',
                                                                          )
    
    timeline, ticks, ticklabels, text = __get_timeaxis(
                                                        utcdatetime=st[0].times(type="utcdatetime"),
                                                        unit='date',
                                                        unitmode='relative',
                                                        dateformat='ddmmyy',
                                                      )    
    
    
    
    ## plotting

    ax[0].plot(timeline, st[0].data, zorder=2, label="E-W", color='k')
    ax[1].plot(timeline, st[1].data, zorder=2,label="N-S", color='k')
    ax[2].plot(timeline, st[2].data, zorder=2, label="int. Temperature", color='darkred')
    ax[3].plot(timeline_furt, FURT['T'], zorder=2, label="FURT T", color='darkred')
    ax[4].plot(timeline_furt, FURT['P'], zorder=2, label="FURT P", color='purple') 
#     ax[5].semilogy(timeline_furt, FURT['Rc'], zorder=2, label="FURT Rain", color='darkblue') 
    ax[5].fill_between(timeline_furt, FURT['Rc'], zorder=2, color='blue', alpha=0.6, label="FURT Rain") 

    
    ax[0].set_ylabel("$\Omega$ (rad)")
    ax[1].set_ylabel("$\Omega$ (rad)")
    ax[2].set_ylabel("T (°C)")
    ax[3].set_ylabel("T (°C)")
    ax[4].set_ylabel("P (hPa)")
    ax[5].set_ylabel("Rain (mm)")
    
    ax[N-1].set_xticklabels(ticklabels[::8], fontsize=font-2)
    ax[N-1].set_xticks(ticks[::8])
    ax[N-1].set_xlabel(text, fontsize=font)
    ax[N-1].set_xlim(timeline[0], timeline[-1])
    
    ax[5].set_ylim(0, nanmax(FURT['Rc']))
    
    for i in range(N):
        ax[i].legend(loc="upper right")
        ax[i].grid(True, ls="--", alpha=0.7, zorder=0)
        

    plt.show();
    return fig,
    
__makeplote_platform(pt, FURT);

## Tilt Vector

In [ ]:
def __get_movement(N, E, Twin=3600):

    from numpy import median, sqrt, arctan, degrees, mod, array, append, rad2deg, nan
    from numpy import ma
    
    dt = N[0].stats.delta
#     t1 = N[0].stats.starttime
    t1 = 0

    Ndata = N[0].data
    Edata = E[0].data

    if ma.is_masked(Ndata):
        Ndata = ma.filled(Ndata, fill_value=nan)
    if ma.is_masked(Edata):
        Edata = ma.filled(Edata, fill_value=nan)
        
    vnorm, vdir, timeline = [], [], []
    while (t1 + Twin) < N[0].stats.npts*dt:
        
        Nmed = median(Ndata[int(t1/dt):int((t1+Twin)/dt)])
        Emed = median(Edata[int(t1/dt):int((t1+Twin)/dt)])

        
        vnorm.append(sqrt(Nmed**2 + Emed**2))
        
        if Nmed > 0 and Emed > 0:
            vdir.append(90-abs(rad2deg(arctan(abs(Nmed/Emed)))))
        
        elif Nmed < 0 and Emed > 0:
            vdir.append(90+abs(rad2deg(arctan(abs(Nmed/Emed)))))
        
        elif Nmed < 0 and Emed < 0:
            vdir.append(270-abs(rad2deg(arctan(abs(Nmed/Emed)))))
        
        elif Nmed > 0 and Emed < 0:
            vdir.append(270+abs(rad2deg(arctan(abs(Nmed/Emed)))))
        
        else:
            vdir.append(nan)
            
        t1 += Twin/2
    
        timeline.append(t1)
    
    time_offset = N[0].stats.starttime.julday
    timeline = array(timeline)/3600 /24 + time_offset# in days
    
    return timeline, array(vdir), array(vnorm)

In [ ]:
def __makeplot_normal_vector_movement(vdirH, vnormH, vdirV, vnormV, vtimelineH, vtimelineV):
    
    #-- Plot... ------------------------------------------------
    fig, ax = plt.subplots(1, 2, subplot_kw=dict(projection='polar'), figsize=(15,5))

    font=12


    p1 = ax[0].scatter(deg2rad(vdirH), vnormH, c=vtimelineH, cmap='plasma', alpha=0.75, s=4, zorder=2)
    p2 = ax[1].scatter(deg2rad(vdirV), vnormV, c=vtimelineV, cmap='plasma', alpha=0.75, s=4, zorder=2)

    cbar1 = plt.colorbar(p1, ax=ax[0], pad=0.1)
    cbar1.set_label('Time in days', rotation=270, fontsize=font, labelpad=18)

    cbar2 = plt.colorbar(p2, ax=ax[1],  pad=0.1)
    cbar2.set_label('Time in days of 2021', rotation=270, fontsize=font, labelpad=18)


#     ax[0].set_ylim(min(vnormH)-0.1*min(vnormH), max(vnormH)+0.05*max(vnormH))

#     ax[1].set_ylim(min(vnormV)-0.05*min(vnormV), max(vnormV)+0.01*max(vnormV))

    ax[0].set_ylim(0.15, 0.18)
    ax[1].set_ylim(0.2, 0.24)

    ax[0].set_theta_zero_location('N')
    ax[0].set_theta_direction(-1)

    ax[1].set_theta_zero_location('N')
    ax[1].set_theta_direction(-1)
    
    ax[0].text(deg2rad(25),93,r"$\mu$rad")
    ax[1].text(deg2rad(25),93,r"$\mu$rad")
    
#     ax[0].set_rgrids(arange(15, 50, 5), angle=60., zorder=0)
#     ax[1].set_rgrids(arange(22.5, 40.0, 2.5), angle=60., zorder=0)

    ax[0].set_title("Platform Tiltmeter",fontsize=font)
    ax[1].set_title("Borehole Tiltmeter",fontsize=font)

In [ ]:
def __makeplot_vector(vdirH, vnormH, vdirV, vnormV, vtimelineH, vtimelineV):
    
    N = 2
    font = 12
 
    fig, ax = plt.subplots(N, 2, figsize=(15,6), sharex=True)
    
#     ax[0][0].plot(vtimelineH, vnormH)
    ax[0][0].scatter(vtimelineH, vnormH, c=vtimelineH, cmap="plasma", s=0.7)
    
#     ax[0][1].plot(vtimelineV, vnormV)
    ax[0][1].scatter(vtimelineV, vnormV, c=vtimelineH, cmap="plasma", s=0.7)
    
#     ax[1][0].plot(vtimelineH, vdirH)
    ax[1][0].scatter(vtimelineH, vdirH, c=vtimelineH, cmap="plasma", s=0.7)

#     ax[1][1].plot(vtimelineV, vdirV)
    ax[1][1].scatter(vtimelineV, vdirV, c=vtimelineH, cmap="plasma", s=0.7)
   
    ax[0][0].grid(zorder=0, alpha=0.7, color="grey", ls=":")
    ax[0][1].grid(zorder=0, alpha=0.7, color="grey", ls=":")
    ax[1][0].grid(zorder=0, alpha=0.7, color="grey", ls=":")
    ax[1][1].grid(zorder=0, alpha=0.7, color="grey", ls=":")
    
    ax[0][0].set_title('Platform Tiltmeter', fontsize=font)
    ax[0][1].set_title('Borehole Tiltmeter', fontsize=font)
    
    ax[0][0].set_ylabel("Vector Norm (mrad)", fontsize=font)
    ax[1][0].set_ylabel("Degrees (°)", fontsize=font)
    
    ax[N-1][0].set_xlabel("Day of Year", fontsize=font)
    ax[N-1][1].set_xlabel("Day of Year", fontsize=font)
    
    return fig

In [ ]:
from numpy import deg2rad

vtimelineH, vdirH, vnormH = __get_movement(pt.select(channel="*N"), pt.select(channel="*E"), Twin=3600)
vtimelineV, vdirV, vnormV = __get_movement(bt.select(channel="*N"), bt.select(channel="*E"), Twin=3600)

vnormH *= 1e3 # convert to milli rad
vnormV *= 1e3 # convert to milli rad

In [ ]:
__makeplot_normal_vector_movement(vdirH, vnormH, vdirV, vnormV, vtimelineH, vtimelineV)

In [ ]:
__makeplot_vector(vdirH, vnormH, vdirV, vnormV, vtimelineH, vtimelineV);

In [ ]:
from obspy.signal.freqattributes import spectrum
from numpy import hanning
from andbro__fft import __fft

import matplotlib.pyplot as plt
import obspy

In [ ]:

datapath = f"/home/andbro/freenas/romy_archive/2022/BW/ROMYT/MAN.D/*"

N = obspy.read(datapath);
N.merge(fill_value='interpolate')

datapath = f"/home/andbro/freenas/romy_archive/2022/BW/ROMYT/MAE.D/*"

E = obspy.read(datapath);
E.merge(fill_value='interpolate')


In [ ]:
TRII = obspy.Stream()
TRII += N
TRII += E

In [ ]:
TRII.plot();

In [ ]:
TRII

In [ ]:
def __makeplotStreamPSD(st, config):

    from scipy import fftpack
    from numpy import abs
    from scipy.signal import welch, hann


    nblock =  4096*100
    overlap = 1028
    win = hann(nblock, True)

    fig, axes = plt.subplots(len(st),2,figsize=(15,10), sharex='col')

    plt.subplots_adjust(hspace=0.3)

    ## _______________________________________________

    st.sort(keys=['channel'], reverse=True)
    
    for i, tr in enumerate(st):

        ff, Pxx = welch(tr.data, fs=tr.stats.sampling_rate, 
                        window=win, noverlap=overlap, nfft=nblock,
                        scaling="density",
                        return_onesided=True)

        ## _________________________________________________________________
        axes[i,0].plot(
                    tr.times(),
                    tr.data,
                    color='black',
                    label='{} {}'.format(tr.stats.station, tr.stats.channel),
                    lw=1.0,
                    )


        ## _________________________________________________________________
        axes[i,1].loglog(
                    ff,
                    Pxx,
                    color='black',
                    lw=1.0,
                    )


        
        axes[i,0].set_ylabel(r'$\Omega$ (rad/s)')    
        axes[i,1].set_ylabel(r'PSD (rad$^2$/Hz)')        
        axes[i,0].legend(loc='upper left',bbox_to_anchor=(0.8, 1.10), framealpha=1.0)
        
#         axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#         axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

        axes[i,1].set_xlim(1e-7,0.25)
#         axes[i,1].set_ylim(1e-12,1e-9)

    return fig

In [ ]:
config = {}

__makeplotStreamPSD(TRII, config);